# Building a Retrieval-Augmented Generation (RAG) System using Pytorch

## Learning Objectives
### Remember
- Define the key components of a RAG system
- Identify the essential libraries required for building a RAG pipeline
- List the main steps in the RAG process

### Understand
- Explain how document embedding works
- Describe the role of vector databases in RAG
- Interpret the relationship between chunks, embeddings, and retrieval

### Apply
- Implement a document loading and chunking pipeline
- Configure a language model for generation
- Set up a vector store for document retrieval

### Analyze
- Compare different chunking strategies
- Examine the impact of various parameter settings
- Debug common issues in RAG systems

### Evaluate
- Assess the quality of generated responses
- Test system performance with different configurations
- Judge the effectiveness of retrieval strategies

### Create
- Design a complete RAG pipeline
- Develop custom prompt templates
- Build an interactive question-answering system

## Prerequisites
- Basic understanding of Python
- Familiarity with machine learning concepts
- Understanding of basic NLP terminology

Let's begin our journey into building a RAG system!

# What is RAG (Retrieval-augmented generation)

**Retrieval-augmented generation (RAG)** is a technique for augmenting LLM knowledge with additional, often private or real-time, data. LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs. The process of bringing the appropriate information and inserting it into the model prompt is known as Retrieval Augmented Generation (RAG).

In [ ]:
!pip install -r rag/requirements.txt

## Run QA over Document

Now, when model created, we can setup Chatbot

A typical RAG application has two main components:

- **Indexing**: a pipeline for ingesting data from a source and indexing it. This usually happen offline.

- **Retrieval and generation**: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

The most common full sequence from raw data to answer looks like:

**Indexing**

1. `Load`: First we need to load our data. We’ll use DocumentLoaders for this.
2. `Split`: Text splitters break large Documents into smaller chunks. This is useful both for indexing data and for passing it in to a model, since large chunks are harder to search over and won’t in a model’s finite context window.
3. `Store`: We need somewhere to store and index our splits, so that they can later be searched over. This is often done using a VectorStore and Embeddings model.

![Indexing pipeline](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/dfed2ba3-0c3a-4e0e-a2a7-01638730486a)

**Retrieval and generation**

1. `Retrieve`: Given a user input, relevant splits are retrieved from storage using a Retriever.
2. `Generate`: A LLM produces an answer using a prompt that includes the question and the retrieved data.

![Retrieval and generation pipeline](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/f0545ddc-c0cd-4569-8c86-9879fdab105a)


## 1. Setting Up the Environment

First, let's import all necessary libraries. We'll go through each import and understand its role in our RAG system.

Key Components:
- LangChain: Framework for developing applications powered by language models
- HuggingFace: Platform for accessing pre-trained models and embeddings
- Chroma: Vector store for efficient similarity search

In [ ]:
from langchain import chains, text_splitter, PromptTemplate
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_community import document_loaders, embeddings, vectorstores
from langchain.llms.base import LLM
from typing import Any, List, Optional, Dict
from pydantic import Field
import torch
import shutil
import intel_extension_for_pytorch as ipex
import time
import os
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer
import ipywidgets as widgets
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

VECTOR_DB_DIR = "vector_dbs"

In [ ]:
class LoadLLM(LLM):
    model_path: str = Field(default="meta-llama/Llama-2-7b-chat-hf")
    model: Optional[Any] = Field(default=None, exclude=True)
    tokenizer: Optional[Any] = Field(default=None, exclude=True)

    class Config:
        arbitrary_types_allowed = True

    def __init__(self, model_path: str = "meta-llama/Llama-2-7b-chat-hf", **kwargs):
        super().__init__(**kwargs)
        self.model_path = model_path
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_path,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        self.model = self.model.to('xpu')
        self.model = ipex.optimize(self.model)
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path, trust_remote_code=True)

    @property
    def _llm_type(self) -> str:
        return "intel_llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        with torch.inference_mode():
            input_ids = self.tokenizer(prompt, return_tensors="pt").to('xpu')
            output = self.model.generate(
                **input_ids,
                do_sample=True,
                max_new_tokens=500
            )
            torch.xpu.synchronize()
            return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> Dict[str, Any]:
        return {"model_path": self.model_path}




## 3. Document Loading and Processing

A crucial part of any RAG system is how it handles document loading and processing. We'll create functions to:
1. Load documents from URLs
2. Split documents into manageable chunks
3. Initialize our embedding model

Understanding these processes is crucial as they directly impact the quality of our retrieval system.

* Document loaders in RAG are used to load and preprocess the documents that will be used for retrieval during the question answering process.
* Document loaders are responsible for preprocessing the documents. This includes tokenizing the text, converting it to the format expected by the retriever, and creating batches of documents.
* Document loaders work in conjunction with the retriever in RAG. The retriever uses the documents loaded by the document loader to find the most relevant documents for a given query.
* The WebBaseLoader in Retrieval Augmented Generation (RAG) is a type of document loader that is designed to load documents from the web.
* The WebBaseLoader is used when the documents for retrieval are not stored locally or in a Hugging Face dataset, but are instead located on the web. This can be useful when you want to use the most up-to-date information available on the internet for your question answering system

### Text splitter

* RecursiveCharacterTextSplitter is used to split text into smaller pieces recursively at the character level. 
* split_documents fuctions splits larger documents into smaller chunks, for easier processing

In [ ]:
def load_document(url):
    print("Loading document from URL...")
    loader = document_loaders.WebBaseLoader(url)
    return loader.load()

def split_document(text, chunk_size=3000, overlap=200):
    print("Splitting document into chunks...")
    text_splitter_instance = text_splitter.RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter_instance.split_documents(text)

def initialize_embedding_fn(embedding_type="huggingface"):
    print(f"Initializing {embedding_type} embeddings...")
    if embedding_type == "huggingface":
        return embeddings.HuggingFaceEmbeddings(
            model_name="sentence-transformers/paraphrase-MiniLM-L3-v2")
    elif embedding_type == "fastembed":
        return FastEmbedEmbeddings(threads=16)
    else:
        raise ValueError(f"Unsupported embedding type: {embedding_type}")

def clear_vector_store(persist_dir=VECTOR_DB_DIR):
    import shutil
    if os.path.exists(persist_dir):
        try:
            # Create a temporary Chroma instance to properly close any open connections
            temp_db = vectorstores.Chroma(
                persist_directory=persist_dir,
                embedding_function=FastEmbedEmbeddings()
            )
            temp_db.persist()
            del temp_db  # Explicitly delete the instance
            
            # Add a small delay to ensure connections are closed
            time.sleep(1)
            
            shutil.rmtree(persist_dir)
        except Exception as e:
            print(f"Warning: Error while clearing vector store: {e}")
            # If regular deletion fails, try forcing it
            os.system(f"rm -rf {persist_dir}")



## 4. Vector Store Management

The vector store is a crucial component in RAG systems. It enables efficient similarity search over document embeddings, allowing us to retrieve relevant context for any given query.

Key Concepts:
- Vector databases
- Persistence and caching
- Similarity search
- Document retrieval
- In Retrieval Augmented Generation (RAG) embeddings play a crucial role in the retrieval of relevant documents for a given query.

* In RAG, each document in the knowledge base is represented as a dense vector, also known as an embedding. These embeddings are typically generated by a transformer model.
* When a query is received, it is also converted into an embedding using the same transformer model. This ensures that the query and the documents are in the same vector space, making it possible to compare them.
* Retrieval: The retrieval step in RAG involves finding the documents whose embeddings are most similar to the query embedding. This is typically done using a nearest neighbor search.


In [ ]:
def get_or_create_embeddings(document_url, embedding_fn, persist_dir=VECTOR_DB_DIR):
    # Create a unique directory name based on timestamp
    import datetime
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    unique_persist_dir = os.path.join(persist_dir, timestamp)
    
    os.makedirs(unique_persist_dir, exist_ok=True)
    
    start_time = time.time()
    print("Creating new vector store...")
    document = load_document(document_url)
    documents = split_document(document)
    
    vector_store = vectorstores.Chroma.from_documents(
        documents=documents,
        embedding=embedding_fn,
        persist_directory=unique_persist_dir,
        collection_metadata={"hnsw:space": "cosine"}
    )
    vector_store.persist()
    print(f"Embedding time: {time.time() - start_time:.2f} seconds")
    return vector_store, unique_persist_dir

## 6. Question-Answering Interface

The question-answering interface provides the user interaction layer of our RAG system. It handles:
1. User input processing
2. Response generation
3. Error handling
4. Performance monitoring

In [ ]:
def handle_user_interaction(vector_store, llm, question):
    prompt_template = """
    Use the following pieces of context to answer the question at the end.
    If you do not know the answer, answer 'I don't know', limit your response to the answer and nothing more.

    {context}

    Question: {question}
    """
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"])
    
    chain_type_kwargs = {"prompt": prompt}
    retriever = vector_store.as_retriever(search_kwargs={"k": 4})
    
    qachain = chains.RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs=chain_type_kwargs,
        return_source_documents=False
    )

    start_time = time.time()
    response = qachain.invoke({"query": question})
    # The response is a dict with 'result' key containing the actual answer
    return response['result'] if isinstance(response, dict) and 'result' in response else response


## 7. Main Application Logic

The main function orchestrates all components of our RAG system. It:
1. Initializes components
2. Sets up the RAG pipeline
3. Manages the interaction loop
4. Handles errors and cleanup

### Retrievers

* Retrievers are responsible for fetching relevant documents from a document store or knowledge base given a query. The retrieved documents are then used by the generator to produce a response.
* RetrievalQA is a type of question answering system that uses a retriever to fetch relevant documents given a question, and then uses a reader to extract the answer from the retrieved documents.
* RetrievalQA can be seen as a two-step process:
    * Retrieval: The retriever fetches relevant documents from the document store given a query.    
    * Generation: The generator uses the retrieved documents to generate a response.
* This two-step process allows RAG to leverage the strengths of both retrieval-based and generation-based approaches to question answering. The retriever allows RAG to efficiently search a large document store, while the generator allows RAG to generate detailed and coherent responses.


In [ ]:
#from langchain_community.vectorstores.chroma import Chroma
def main(document_url, question, embedding_type="huggingface", model_path="meta-llama/Llama-2-7b-chat-hf"):
    try:
        llm = LoadLLM(model_path=model_path)
        embedding_fn = initialize_embedding_fn(embedding_type)
        vector_store, persist_dir = get_or_create_embeddings(document_url, embedding_fn)
        answer = handle_user_interaction(vector_store, llm, question)
        vector_store.persist()
        del vector_store
        # Cleanup old directories if needed
        cleanup_old_directories(VECTOR_DB_DIR, keep_last=5)
        return answer
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def cleanup_old_directories(base_dir, keep_last=5):
    """Clean up old vector store directories, keeping only the most recent ones."""
    try:
        dirs = [os.path.join(base_dir, d) for d in os.listdir(base_dir)]
        dirs = [d for d in dirs if os.path.isdir(d)]
        dirs.sort(key=lambda x: os.path.getctime(x), reverse=True)
        
        # Remove all but the last keep_last directories
        for dir_path in dirs[keep_last:]:
            try:
                shutil.rmtree(dir_path)
            except Exception as e:
                print(f"Warning: Could not remove directory {dir_path}: {e}")
    except Exception as e:
        print(f"Warning: Error during cleanup: {e}")

### Run QA with Ipywidgets

In [ ]:
# Create input widgets
url_input = widgets.Text(
    value='',
    placeholder='Enter URL here',
    description='URL:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

question_input = widgets.Text(
    value='',
    placeholder='Enter your question here',
    description='Question:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

submit_button = widgets.Button(
    description='Get Answer',
    button_style='primary',
    layout=widgets.Layout(width='20%')
)

output_area = widgets.Output()

def on_submit_button_clicked(b):
    with output_area:
        clear_output()
        if not url_input.value or not question_input.value:
            print("Please enter both URL and question.")
            return
        
        response = main(url_input.value, question_input.value)
        print(response)  # Now this will be just the answer text

submit_button.on_click(on_submit_button_clicked)

# Display the widgets
display(url_input)
display(question_input)
display(submit_button)
display(output_area)

## Workshop Summary

In this workshop, we've built a complete RAG system from scratch, covering:

### Technical Components
1. Document Processing
   - Web document loading
   - Text chunking strategies
   - Embedding generation

2. Vector Store Management
   - Persistent storage
   - Efficient retrieval
   - Similarity search

3. Language Model Integration
   - Model configuration
   - Parameter optimization
   - Response generation

### Key Learnings
1. System Architecture
   - Understanding RAG pipeline components
   - Component interaction
   - Error handling

2. Performance Optimization
   - Memory management
   - GPU utilization
   - Response time optimization

3. Best Practices
   - Code organization
   - Documentation
   - Error handling

### Next Steps
To further improve the system, consider:
1. Implementing different embedding models
2. Experimenting with chunk sizes and overlap
3. Adding evaluation metrics
4. Implementing caching mechanisms
5. Adding support for different document types

This workshop provides a foundation for building and understanding RAG systems, which you can extend and customize for your specific use cases.